In [2]:
import pyproj
import glob
import os
import geopandas as gpd
from shapely.geometry import Point

In [3]:
# Define input and output coordinate systems
input_epsg = 'EPSG:4152'  # NAVD88 Geoid18 (latitude and longitude)
output_epsg = 'EPSG:6344'  # NAVD88 Geoid12 (easting and northing)

# Create coordinate transformation function
transformer = pyproj.Transformer.from_crs(input_epsg, output_epsg, always_xy=True)

# Define input and output directories
input_directory = 'F:\\EastOilFields\\xyz\\2006'  # Directory containing input XYZ files
output_directory = 'F:\\EastOilFields\\shp\\2006shapefilez'  # Directory to store output shapefiles

# Get list of XYZ files in the input directory
xyz_files = glob.glob(os.path.join(input_directory, '*.xyz'))

In [4]:
# Iterate over XYZ files and perform the conversion
for xyz_file in xyz_files:
    # Read XYZ file
    data = []
    with open(xyz_file, 'r') as f:
        for line in f:
            x, y, z = line.strip().split()  # Assuming X, Y, Z values are space-separated
            data.append((float(x), float(y), float(z)))

    # Convert XYZ coordinates to Point objects and transform to the output coordinate system
    points = []
    for x, y, z in data:
        point = Point(x, y)
        transformed_point = transformer.transform(point.x, point.y, z)
        points.append((transformed_point[0], transformed_point[1], z))  # Append "z" value

    # Create a GeoDataFrame with transformed coordinates and "z" values
    df = gpd.GeoDataFrame(points, columns=['x', 'y', 'z'], geometry=gpd.points_from_xy(*zip(*points)))

    # Create output file name
    output_file = os.path.splitext(os.path.basename(xyz_file))[0] + '.shp'
    output_path = os.path.join(output_directory, output_file)

    # Save GeoDataFrame as shapefile
    df.to_file(output_path)
